In [17]:
!pip install --upgrade pandas tqdm openpyxl

Requirement already up-to-date: pandas in ./venv/lib/python3.8/site-packages (1.3.1)
Requirement already up-to-date: tqdm in ./venv/lib/python3.8/site-packages (4.62.0)
     |████████████████████████████████| 243 kB 4.6 MB/s 


In [80]:
import os
from tqdm.notebook import tqdm

In [81]:
# copypasta from tqdm docs
class TqdmUpTo(tqdm):
    """Provides `update_to(n)` which uses `tqdm.update(delta_n)`."""
    def update_to(self, b=1, bsize=1, tsize=None):
        """
        b  : int, optional
            Number of blocks transferred so far [default: 1].
        bsize  : int, optional
            Size of each block (in tqdm units) [default: 1].
        tsize  : int, optional
            Total size (in tqdm units). If [default: None] remains unchanged.
        """
        if tsize is not None:
            self.total = tsize
        return self.update(b * bsize - self.n)  # also sets self.n = b * bsize

In [82]:
RESULTS_URL="https://www.sos.wa.gov/_assets/elections/research/2020Gen_Precinct_Results_GIS-Ready.xlsx"
# https://stackoverflow.com/questions/39125532/file-does-not-exist-in-jupyter-notebook
RESULTS_FILENAME='data.xlsx'
# https://kingcounty.gov/depts/elections/elections/maps/precinct-and-district-data.aspx
PRECINCTS_URL='https://kingcounty.gov/~/media/depts/elections/elections/maps/precinct-and-district-data.ashx?la=en'
PRECINCTS_FILENAME='precinct-and-district-data.xlsx'
OUT_FILENAME='results_in_seattle.csv'

In [83]:
import urllib.request

def download(url, filename):
  req = urllib.request.Request(
      url, 
      data=None, 
      headers={
          'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
      }
  )
  # copypasta from tqdm docs
  if not os.path.exists(filename):
    with urllib.request.urlopen(req) as response:
      with tqdm.wrapattr(open(filename, "wb"), "write",
                        miniters=1, desc=url.split('/')[-1],
                        total=getattr(response, 'length', None)) as fout:
        for chunk in response:
            fout.write(chunk)

In [84]:
download(RESULTS_URL, RESULTS_FILENAME)
download(PRECINCTS_URL, PRECINCTS_FILENAME)

2020Gen_Precinct_Results_GIS-Ready.xlsx:   0%|          | 0/63189914 [00:00<?, ?it/s]

precinct-and-district-data.ashx?la=en:   0%|          | 0/111155 [00:00<?, ?it/s]

In [85]:
import pandas

with open(RESULTS_FILENAME, 'rb') as f:
  results = pandas.read_excel(f)
with open(PRECINCTS_FILENAME, 'rb') as f:
  precincts = pandas.read_excel(f)

In [86]:
# just king country results
results = results[results.County == 'King']
# unique district names
results['DistrictName'].unique()


array(['Turnout', 'State of Washington', 'United States',
       'Congressional District 7', 'Legislative District 32',
       'Supreme Court', 'Court of Appeals, Division 1, District 1',
       'King Superior Court', 'Legislative District 46',
       'Legislative District 33', 'Congressional District 1',
       'Legislative District 45', 'Congressional District 9',
       'Legislative District 41', 'Congressional District 8',
       'Legislative District 30', 'Legislative District 1',
       'Legislative District 37', 'Legislative District 34',
       'Legislative District 47', 'Legislative District 11',
       'Legislative District 5', 'Legislative District 31',
       'Legislative District 48', 'Legislative District 39',
       'Legislative District 43', 'Legislative District 36'], dtype=object)

In [87]:
# just seattle precincts
precincts = precincts[precincts.SeattleCouncilDistrict.notnull()]
precincts

,PrecinctCode,PrecinctName,CountyCouncil,LegislativeDistrict,CongressionalDistrict,SeattleCouncilDistrict
761,1245,SEA 34-1245,8,34,7,1.0
762,1246,SEA 34-1246,8,34,7,1.0
763,1247,SEA 34-1247,8,34,7,1.0
764,1248,SEA 34-1248,8,34,7,1.0
765,1249,SEA 34-1249,8,34,7,1.0
...,...,...,...,...,...,...
2741,3948,SEA 37-3948,2,37,9,2.0
2742,3949,SEA 43-3949,2,43,7,3.0
2743,3950,SEA 43-3950,2,43,7,4.0
2744,3951,SEA 43-3951,4,43,7,4.0


In [88]:
# unique precinct names
seattle_precincts = precincts['PrecinctName'].unique()
seattle_precincts

array(['SEA 34-1245', 'SEA 34-1246', 'SEA 34-1247', ..., 'SEA 43-3950',
       'SEA 43-3951', 'SEA 43-3952'], dtype=object)

In [89]:
results_in_seattle = results[results.PrecinctName.isin(seattle_precincts)]
results_in_seattle

,DistrictName,RaceName,SeqNo,Candidate,BallotOrder,PrecinctCode,PrecinctName,County,CandCode,Votes
156417,Turnout,Turnout,0,Registered Voters,1,KI00001245,SEA 34-1245,King,G20TREGVOT,571
156418,Turnout,Turnout,0,Ballots Cast,2,KI00001245,SEA 34-1245,King,G20TBALCST,500
156419,State of Washington,Referendum Measure No. 90,200,Approved,1,KI00001245,SEA 34-1245,King,G20R0090A,386
156420,State of Washington,Referendum Measure No. 90,200,Rejected,2,KI00001245,SEA 34-1245,King,G20R0090R,85
156421,State of Washington,Advisory Vote No. 32,632,Repealed,1,KI00001245,SEA 34-1245,King,G20A0032R,184
...,...,...,...,...,...,...,...,...,...,...
301080,King Superior Court,Judge Position 13,11713,Andrea Robertson,2,KI00003840,SEA 37-3840,King,G20KI13ROB,160
301081,King Superior Court,Judge Position 13,11713,WRITE-IN,3,KI00003840,SEA 37-3840,King,G20KI13W00,4
301082,King Superior Court,Judge Position 30,11730,Doug North,1,KI00003840,SEA 37-3840,King,G20KI30NOR,161
301083,King Superior Court,Judge Position 30,11730,Carolyn Ladd,2,KI00003840,SEA 37-3840,King,G20KI30LAD,204


In [90]:
with open(OUT_FILENAME, 'wb') as f:
  results_in_seattle.to_csv(f)